In [104]:
import pandas as pd
import numpy as np
from numpy import NAN
from subprocess import check_output
from IPython.display import display
pd.options.display.max_columns=None
from Preprocessing.read_file import read
from Preprocessing.preprocess import makefile
from sklearn.model_selection import KFold
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report

In [33]:
df_2017 = pd.read_csv("./data_2017.csv")
df_2018 = pd.read_csv("./data_2018.csv")

In [34]:
# Dimensionality Reduction based on correlation analysis
df = reduce(df_2017, 2017)
df_2018 = reduce(df_2018, 2018)
df = df.filter(items = df_2018.columns)

===== Start reducing features ======
===== Finish reducing features ======
===== Start reducing features ======
===== Finish reducing features ======


In [35]:
# Scamble and subset data frame into train + validation(80%) and test(10%)
df = df.sample(frac=1).reset_index(drop=True)
split_ratio = 0.8
print('Train and Test Split Ratio : ', split_ratio)
df_train = df[ : int(len(df) * split_ratio)]
df_test = df[int(len(df) * split_ratio) : ]

Train and Test Split Ratio :  0.8


In [36]:
df_train.head()

,2nd Color Variation (Dark Yellow +),Light Green,Stem Area,Flatness,Fruit Center X (mm),Fruit Center Y (mm),LR_Diff,Smoothness,Texture,VisionGrade,Weight,block_Num
0,16.914174,0.0,0.0,11.6373,2.7081,-3.6424,25.4,100.0000,165.2646,0,125.4,64E
1,16.914174,0.0,0.0,21.1073,2.6584,-0.9148,13.3,100.0000,324.9619,0,167.5,64E
2,16.914174,0.0,0.0,10.0975,5.1768,-2.2727,28.5,100.0000,321.4305,0,115.5,64E
3,16.914174,0.0,0.0,17.6913,4.2401,-2.6561,30.5,93.1888,1178.2140,2,157.9,64E
4,16.914174,0.0,0.0,10.6962,3.9978,-2.3516,24.3,100.0000,832.8530,2,116.8,64E


In [5]:
# batch_info = df.iloc[[2], 0:10] # List [2] returns dataframe, 2 returns object
# batch_info = batch_info.reset_index(drop=True)
# batch_info
# batch_info.columns = df.iloc[1, 0: 10]
# batch_info.columns.name = NAN
# batch_info = batch_info.dropna()
# batch_info.columns
# batch_info.head()
# batch_info.shape

In [38]:
#fruit_info = fruit_info.dropna(axis = 1)
#print(fruit_info.shape)
df_2017 = df_2017.loc[:, ~df_2017.columns.duplicated()]
print(df_2017.shape)

(218150, 97)


In [39]:
df_2017 = df_2017.rename(index=str, columns={"IQS 1 avg" : "brix", "IQS 2 avg" : "granulation",
                                                      "IQS 3 avg" : "seed_counts", "IQS 4 avg" : "frost_damage"})
#fruit_info = fruit_info[['Grade']]
df_feature = df_2017[['2nd Color Variation (Dark Yellow +)', 'Elongation', 'Fruit', 'brix',
                         'granulation', 'Orange', 'Overall Roundness','Calyx size', 'seed_counts', 
                         'Stem Size', # Fill with mean
                         
                         'Creases', 'Dark Orange', 'frost_damage', 'Lemon (Dark Yellow +)', 
                         'Light Green', 'Orange/Yellow', 'Red', 'Red/Orange', 'Rough skin', 
                         'Ridges', 'Stem', 'Stem Area' , 'Stem Rot', # Fill with zero
                         
                         'Stem angle', 'Trapeziod Angle (Deg)', # Fill with 180
                         
                         'DownWeight', 'Flatness',  'Fruit Center X (mm)', 'Fruit Center Y (mm)', 'Grade', 
                         'LR_Diff', 'Major Diameter (mm)', 'SizerSize', 'Smoothness', 'Start of Batch', 
                         'Texture', 'UpWeight', 'VisionGrade', 'Volume', 'Weight' # No Filling 
                         ]]

df_target = df_2017[['block_Num']]
#df_feature.iloc[20000, :]
df_feature.head(n = 20)


,2nd Color Variation (Dark Yellow +),Elongation,Fruit,brix,granulation,Orange,Overall Roundness,Calyx size,seed_counts,Stem Size,Creases,Dark Orange,frost_damage,Lemon (Dark Yellow +),Light Green,Orange/Yellow,Red,Red/Orange,Rough skin,Ridges,Stem,Stem Area,Stem Rot,Stem angle,Trapeziod Angle (Deg),DownWeight,Flatness,Fruit Center X (mm),Fruit Center Y (mm),Grade,LR_Diff,Major Diameter (mm),SizerSize,Smoothness,Start of Batch,Texture,UpWeight,VisionGrade,Volume,Weight
0,NaN,1.5727,100.0000,14.070,20.515,0.0000,98.4426,48.2911,0.0,722.7693,0.0,0.0000,0.0,NaN,NaN,0.0084,0.0000,0.0000,0.0000,0.0000,NaN,0.0,NaN,142.8835,180.0000,53.75,3.6333,3.6078,-2.5151,D,32.9,73.5655,8,100.0000,93.9544,5497.5776,86.65,3,195.7,140.4
1,NaN,4.6396,11.1111,11.865,31.455,81.9133,95.4993,20.5092,0.0,191.3997,0.0,17.5335,0.0,NaN,NaN,0.1160,0.1328,0.1194,88.8889,0.0000,NaN,0.0,NaN,150.1400,180.0000,144.30,11.2052,0.5329,-0.7941,A,38.6,87.1679,10,100.0000,0.0000,889.5946,182.90,0,342.0,327.2
2,NaN,7.2531,0.0000,10.640,33.290,83.7416,92.9991,231.1212,0.0,428.0046,0.0,10.5472,0.0,NaN,NaN,4.4543,0.1544,0.4562,100.0000,0.0000,NaN,0.0,NaN,180.0000,180.0000,176.30,5.1324,4.2151,-1.9970,A,58.3,96.8204,12,92.9884,0.0000,1209.4880,234.60,0,460.7,410.9
3,NaN,9.4746,7.1429,11.455,35.075,89.5043,90.9543,195.4548,0.0,389.1873,0.0,9.2675,0.0,NaN,NaN,0.3935,0.1245,0.2198,89.2857,3.5714,NaN,0.0,NaN,180.0000,180.0000,156.45,4.1777,2.7572,-4.8641,A,16.3,88.6671,11,95.0483,0.0000,839.0543,172.75,0,371.9,329.2
4,NaN,10.8893,0.0000,11.570,32.330,87.4088,89.6720,115.9858,0.0,411.0793,0.0,10.6438,0.0,NaN,NaN,1.4789,0.1557,0.2076,100.0000,0.0000,NaN,0.0,NaN,173.5012,180.0000,145.15,6.6170,4.0348,-3.8334,A,46.9,90.9928,11,93.3853,0.0000,918.8754,192.05,0,382.6,337.2
5,NaN,5.2413,3.8462,11.520,31.950,89.4532,94.9308,2.4786,0.0,600.0881,0.0,9.1715,0.0,NaN,NaN,0.6172,0.0643,0.1669,96.1538,0.0000,NaN,0.0,NaN,153.5318,180.0000,132.80,7.6472,3.0177,-2.7885,A,20.2,84.6418,10,94.0734,0.0070,989.3513,153.00,0,318.0,285.8
6,NaN,6.5517,29.6296,11.530,33.210,91.6975,93.6552,76.5134,0.0,303.5162,0.0,6.8798,0.0,NaN,NaN,0.9539,0.0675,0.1550,70.3704,0.0000,NaN,0.0,NaN,176.7582,180.0000,102.60,5.5621,2.1808,-0.6472,A,28.7,79.9787,9,92.5168,0.0000,607.2122,131.30,0,255.9,233.9
7,NaN,3.4355,15.3846,11.270,33.690,87.6128,96.6287,3.4998,0.0,363.4999,0.0,9.9428,0.0,NaN,NaN,2.0082,0.0983,0.1459,84.6154,0.0000,NaN,0.0,NaN,131.6743,180.0000,192.30,5.7389,3.2340,-0.1210,A,20.2,95.7197,12,92.4360,0.0000,533.2753,212.50,0,448.6,404.8
8,NaN,2.7428,23.0769,10.920,33.840,82.7556,97.3056,1.3687,0.0,330.5579,0.0,13.7921,0.0,NaN,NaN,2.3327,0.3239,0.3161,76.9231,0.0000,NaN,0.0,NaN,145.4389,180.0000,157.45,4.6802,5.0319,-5.5565,A,58.5,96.1967,12,93.1983,0.0000,1025.9015,215.95,0,402.7,373.4
9,NaN,2.8609,59.2593,11.690,31.395,90.0982,97.1737,268.0507,0.0,369.8641,0.0,7.8109,0.0,NaN,NaN,1.0956,0.0718,0.2906,40.7407,0.0000,NaN,0.0,NaN,154.5077,160.7371,148.65,6.7421,2.3388,0.5317,A,28.8,88.8219,11,94.3320,0.0017,655.5113,177.45,0,329.2,326.1


In [40]:
df_feature = df_feature.fillna(df_feature.mean()['2nd Color Variation (Dark Yellow +)':'Stem Size'])
df_feature.iloc[20000, :]

2nd Color Variation (Dark Yellow +)     8.0114
Elongation                             5.14438
Fruit                                  71.6392
brix                                   12.2732
granulation                            77.4864
Orange                                  51.772
Overall Roundness                      94.0533
Calyx size                             7.14693
seed_counts                            6.09025
Stem Size                              38.3413
Creases                                    NaN
Dark Orange                                NaN
frost_damage                               NaN
Lemon (Dark Yellow +)                  38.2555
Light Green                            99.5259
Orange/Yellow                              NaN
Red                                        NaN
Red/Orange                                 NaN
Rough skin                                 NaN
Ridges                                     NaN
Stem                                       NaN
Stem Area    

In [41]:
df_feature.loc[:, 'Creases' : 'Stem Rot'] = df_feature.loc[:, 'Creases' : 'Stem Rot'].fillna(0)
df_feature.loc[:, 'Stem angle' : 'Trapeziod Angle (Deg)'] = df_feature.loc[:, 'Stem angle' : 'Trapeziod Angle (Deg)'].fillna(180)

df_feature.iloc[20000, :]

2nd Color Variation (Dark Yellow +)     8.0114
Elongation                             5.14438
Fruit                                  71.6392
brix                                   12.2732
granulation                            77.4864
Orange                                  51.772
Overall Roundness                      94.0533
Calyx size                             7.14693
seed_counts                            6.09025
Stem Size                              38.3413
Creases                                      0
Dark Orange                                  0
frost_damage                                 0
Lemon (Dark Yellow +)                  38.2555
Light Green                            99.5259
Orange/Yellow                                0
Red                                          0
Red/Orange                                   0
Rough skin                                   0
Ridges                                       0
Stem                                         0
Stem Area    

In [42]:
df_feature.Grade = [ord(x) - 64 for x in df_feature.Grade]
df_feature.Grade

0         4
1         1
2         1
3         1
4         1
5         1
6         1
7         1
8         1
9         1
10        1
11        1
12        1
13        1
14        1
15        1
16        1
17        1
18        1
19        1
20        1
21        1
22        1
23        1
24        1
25        1
26        1
27        1
28        1
29        1
         ..
218120    1
218121    1
218122    1
218123    1
218124    1
218125    1
218126    1
218127    1
218128    1
218129    1
218130    1
218131    1
218132    1
218133    1
218134    1
218135    1
218136    1
218137    1
218138    1
218139    1
218140    1
218141    1
218142    1
218143    1
218144    1
218145    1
218146    1
218147    1
218148    1
218149    1
Name: Grade, Length: 218150, dtype: int64

In [39]:
# df_weight = fruit_info[['Weight', 'UpWeight', 'DownWeight']]
# df_weight.astype('float64').corr()
# fruit_info[['MajorDiameter', 'MinorDiameter']].astype('float64').corr()

* Investigate the correlation among features

In [43]:
correlation_matrix = df_feature.astype('float64').corr()
correlation_matrix

,2nd Color Variation (Dark Yellow +),Elongation,Fruit,brix,granulation,Orange,Overall Roundness,Calyx size,seed_counts,Stem Size,Creases,Dark Orange,frost_damage,Lemon (Dark Yellow +),Light Green,Orange/Yellow,Red,Red/Orange,Rough skin,Ridges,Stem,Stem Area,Stem Rot,Stem angle,Trapeziod Angle (Deg),DownWeight,Flatness,Fruit Center X (mm),Fruit Center Y (mm),Grade,LR_Diff,Major Diameter (mm),SizerSize,Smoothness,Start of Batch,Texture,UpWeight,VisionGrade,Volume,Weight
2nd Color Variation (Dark Yellow +),1.000000e+00,-3.857488e-25,-2.395003e-24,-4.622536e-24,-4.431001e-25,-3.172664e-25,1.496612e-23,1.471807e-24,5.342489e-25,-3.962475e-25,-4.106805e-16,-3.851667e-16,-7.555378e-16,2.225900e-01,-2.500499e-01,-3.957486e-16,-2.348825e-16,-2.125822e-16,-3.926515e-16,-9.888634e-17,-7.286400e-17,-7.518797e-02,-8.274148e-17,2.558383e-16,1.016790e-16,0.038017,0.017880,-0.013942,0.009636,0.019156,-0.007599,0.041222,0.041998,0.003215,0.062674,0.061701,0.030911,0.019156,0.041310,0.035269
Elongation,-3.857488e-25,1.000000e+00,-2.082387e-02,-3.268996e-02,-1.742894e-01,1.226485e-01,-4.929951e-01,-4.245451e-02,-5.502932e-03,-9.383307e-02,1.779355e-02,-1.497015e-01,1.652034e-02,2.461048e-15,2.014387e-15,1.699969e-01,-2.063538e-02,6.429825e-01,2.539477e-02,8.298904e-03,-5.359846e-02,2.724528e-15,-8.154976e-02,1.434307e-01,1.142651e-01,-0.068258,0.401224,0.265583,0.021841,0.235455,-0.065655,0.096481,0.050130,-0.170437,-0.037772,0.006011,-0.091708,0.235455,-0.060284,-0.082882
Fruit,-2.395003e-24,-2.082387e-02,1.000000e+00,3.213166e-01,4.020318e-01,-3.203131e-01,1.772655e-01,-3.314765e-03,-2.238891e-01,8.577381e-02,-2.679988e-01,4.700718e-01,-2.119225e-02,5.738124e-15,3.423920e-14,-1.921815e-01,2.261723e-01,1.393700e-01,-9.127054e-01,-2.104634e-01,7.515219e-02,-5.855135e-14,2.035139e-01,-1.876289e-01,4.388426e-02,-0.438009,-0.241603,0.081577,-0.075819,-0.451820,0.017570,-0.504654,-0.382933,0.069436,0.027688,-0.514311,-0.388093,-0.451820,-0.537735,-0.423604
brix,-4.622536e-24,-3.268996e-02,3.213166e-01,1.000000e+00,1.301810e-01,-1.481194e-01,4.836447e-02,-1.049176e-02,-2.253203e-01,2.676737e-02,-3.392621e-02,2.465324e-01,5.850831e-02,-3.065825e-14,-3.085289e-14,-3.154267e-01,1.741831e-01,-1.472615e-02,-3.086345e-01,-7.828587e-02,6.278998e-02,-3.299716e-14,8.598480e-02,-1.467566e-01,2.933271e-02,-0.210119,-0.068136,-0.003071,-0.063218,-0.250891,0.026709,-0.233818,-0.172934,0.063493,0.102598,-0.123149,-0.177827,-0.250891,-0.252439,-0.198700
granulation,-4.431001e-25,-1.742894e-01,4.020318e-01,1.301810e-01,1.000000e+00,-7.822819e-01,2.137093e-01,2.130973e-01,6.004520e-02,4.716340e-01,-2.309593e-01,8.871489e-01,-6.972178e-02,1.283032e-16,2.443856e-15,-6.350652e-01,4.481763e-01,1.199983e-02,-3.402771e-01,-8.717596e-02,1.995322e-01,-5.986779e-16,4.779153e-01,-5.016048e-01,-6.231759e-02,0.226963,-0.315748,-0.013615,-0.025695,-0.706584,0.246937,0.157600,0.093512,0.052068,-0.027043,-0.266298,0.318007,-0.706584,0.155320,0.282649
Orange,-3.172664e-25,1.226485e-01,-3.203131e-01,-1.481194e-01,-7.822819e-01,1.000000e+00,-9.177172e-02,-8.796794e-02,-1.145128e-01,-2.912441e-01,3.147047e-01,-8.153647e-01,5.709072e-02,-5.098633e-15,-3.349314e-15,5.282743e-01,-5.357867e-01,-1.533581e-02,2.946916e-01,1.096306e-01,-1.921332e-01,-4.814302e-15,-3.814353e-01,4.856972e-01,1.334329e-01,-0.110626,0.307530,0.045345,0.011723,0.562125,-0.193295,-0.057112,-0.025061,0.043896,-0.076230,0.123008,-0.188305,0.562125,-0.067989,-0.155758
Overall Roundness,1.496612e-23,-4.929951e-01,1.772655e-01,4.836447e-02,2.137093e-01,-9.177172e-02,1.000000e+00,8.078001e-02,-5.606616e-03,1.428927e-01,-3.777781e-02,1.986442e-01,-1.800957e-02,1.260510e-13,1.192442e-13,-1.120476e-01,6.843913e-02,-2.759455e-01,-3.148022e-02,-1.662756e-02,6.275445e-02,1.276816e-13,1.029026e-01,4.164285e-01,5.414578e-01,0.091499,-0.295951,-0.012396,-0.079105,-0.199849,0.083260,0.197725,0.026873,0.313528,0.036415,-0.015047,0.120764,-0.199849,0.112032,0.109930
Calyx size,1.471807e-24,-4.245451e-02,-3.314765e-03,-1.049176e-02,2

In [44]:
for col in correlation_matrix.columns :
    if correlation_matrix[col].between(0.8, 1.0, inclusive = False).any() or \
       correlation_matrix[col].between(-1.0, -0.8).any():
        correlation_matrix = correlation_matrix.drop(col)
        correlation_matrix = correlation_matrix.drop(col, axis = 1)
print("shape : " + str(correlation_matrix.shape))
correlation_matrix

shape : (33, 33)


,2nd Color Variation (Dark Yellow +),Elongation,brix,Overall Roundness,Calyx size,seed_counts,Stem Size,Creases,Dark Orange,frost_damage,Lemon (Dark Yellow +),Light Green,Orange/Yellow,Red,Red/Orange,Rough skin,Ridges,Stem,Stem Area,Stem Rot,Stem angle,Trapeziod Angle (Deg),Flatness,Fruit Center X (mm),Fruit Center Y (mm),Grade,LR_Diff,SizerSize,Smoothness,Start of Batch,Texture,VisionGrade,Weight
2nd Color Variation (Dark Yellow +),1.000000e+00,-3.857488e-25,-4.622536e-24,1.496612e-23,1.471807e-24,5.342489e-25,-3.962475e-25,-4.106805e-16,-3.851667e-16,-7.555378e-16,2.225900e-01,-2.500499e-01,-3.957486e-16,-2.348825e-16,-2.125822e-16,-3.926515e-16,-9.888634e-17,-7.286400e-17,-7.518797e-02,-8.274148e-17,2.558383e-16,1.016790e-16,0.017880,-0.013942,0.009636,0.019156,-0.007599,0.041998,0.003215,0.062674,0.061701,0.019156,0.035269
Elongation,-3.857488e-25,1.000000e+00,-3.268996e-02,-4.929951e-01,-4.245451e-02,-5.502932e-03,-9.383307e-02,1.779355e-02,-1.497015e-01,1.652034e-02,2.461048e-15,2.014387e-15,1.699969e-01,-2.063538e-02,6.429825e-01,2.539477e-02,8.298904e-03,-5.359846e-02,2.724528e-15,-8.154976e-02,1.434307e-01,1.142651e-01,0.401224,0.265583,0.021841,0.235455,-0.065655,0.050130,-0.170437,-0.037772,0.006011,0.235455,-0.082882
brix,-4.622536e-24,-3.268996e-02,1.000000e+00,4.836447e-02,-1.049176e-02,-2.253203e-01,2.676737e-02,-3.392621e-02,2.465324e-01,5.850831e-02,-3.065825e-14,-3.085289e-14,-3.154267e-01,1.741831e-01,-1.472615e-02,-3.086345e-01,-7.828587e-02,6.278998e-02,-3.299716e-14,8.598480e-02,-1.467566e-01,2.933271e-02,-0.068136,-0.003071,-0.063218,-0.250891,0.026709,-0.172934,0.063493,0.102598,-0.123149,-0.250891,-0.198700
Overall Roundness,1.496612e-23,-4.929951e-01,4.836447e-02,1.000000e+00,8.078001e-02,-5.606616e-03,1.428927e-01,-3.777781e-02,1.986442e-01,-1.800957e-02,1.260510e-13,1.192442e-13,-1.120476e-01,6.843913e-02,-2.759455e-01,-3.148022e-02,-1.662756e-02,6.275445e-02,1.276816e-13,1.029026e-01,4.164285e-01,5.414578e-01,-0.295951,-0.012396,-0.079105,-0.199849,0.083260,0.026873,0.313528,0.036415,-0.015047,-0.199849,0.109930
Calyx size,1.471807e-24,-4.245451e-02,-1.049176e-02,8.078001e-02,1.000000e+00,-1.965309e-01,6.816989e-01,-1.298076e-01,1.925450e-01,-2.513609e-01,2.956289e-14,3.692944e-14,-2.670556e-01,8.446938e-02,7.125843e-02,4.752113e-02,1.564204e-02,4.009403e-02,-3.707313e-14,9.047753e-02,-2.052417e-01,-1.407062e-02,-0.193375,-0.066883,0.009480,-0.307880,0.098402,0.270066,0.009346,-0.002270,-0.045612,-0.307880,0.436957
seed_counts,5.342489e-25,-5.502932e-03,-2.253203e-01,-5.606616e-03,-1.965309e-01,1.000000e+00,-1.997773e-01,3.829956e-02,-7.269425e-02,2.189427e-01,7.701482e-15,1.321861e-14,7.539118e-02,-3.059335e-02,-7.809347e-02,2.133980e-01,5.319632e-02,1.981474e-01,3.957383e-15,1.064268e-02,8.007236e-02,-7.851735e-03,0.048436,-0.004703,0.025981,0.236657,0.007813,0.087390,-0.029754,0.209617,0.312947,0.236657,0.082119
Stem Size,-3.962475e-25,-9.383307e-02,2.676737e-02,1.428927e-01,6.816989e-01,-1.997773e-01,1.000000e+00,-1.829437e-01,4.249458e-01,-2.643545e-01,5.543913e-14,4.055305e-14,-4.345511e-01,1.988682e-01,8.432477e-02,-1.929433e-02,-1.603135e-02,1.352809e-01,-6.729115e-14,2.351858e-01,-3.685927e-01,-4.377805e-02,-0.279871,-0.068073,0.013213,-0.496508,0.166044,0.321139,0.013205,-0.001217,-0.094015,-0.496508,0.562100
Creases,-4.106805e-16,1.779355e-02,-3.392621e-02,-3.777781e-02,-1.298076e-01,3.829956e-02,-1.829437e-01,1.000000e+00,-2.086497e-01,2.201329e-01,-1.525770e-01,-1.449692e-01,2.472691e-01,-1.624876e-01,-1.625317e-02,1.101875e-01,3.955710e-02,-3.406827e-02,-1.545532e-01,-8.588675e-02,1.364877e-01,7.635292e-02,0.232442,0.012935,-0.015165,0.273814,-0.084324,-0.149478,0.034975,-0.035244,0.023418,0.273814,-0.126837
Dark Orange,-3.851667e-16,-1.497015e-01,2.465324e-01,1.986442e-01,1.925450e-01,-7.269425e-02,4.249458e-01,-2.086497e-01,1.000000e+00,3.947380e-02,-2.252211e-01,-2.139911e-01,-5.760090e-01,6.274970e-01,7.107218e-02,-3.737494e-01,-9.349515e-02,1.667273e-01,-2.281383e-01,4.581422e-01

In [45]:
df_feature_filtered = df_feature[correlation_matrix.columns]
print('Shape : ' + str(df_feature_filtered.shape))
df_feature_filtered = pd.concat([df_feature_filtered, df_target], axis=1)
print('Shape : ' + str(df_feature_filtered.shape))
df_feature_filtered.head()

Shape : (218150, 33)
Shape : (218150, 34)


,2nd Color Variation (Dark Yellow +),Elongation,brix,Overall Roundness,Calyx size,seed_counts,Stem Size,Creases,Dark Orange,frost_damage,Lemon (Dark Yellow +),Light Green,Orange/Yellow,Red,Red/Orange,Rough skin,Ridges,Stem,Stem Area,Stem Rot,Stem angle,Trapeziod Angle (Deg),Flatness,Fruit Center X (mm),Fruit Center Y (mm),Grade,LR_Diff,SizerSize,Smoothness,Start of Batch,Texture,VisionGrade,Weight,block_Num
0,16.914174,1.5727,14.070,98.4426,48.2911,0.0,722.7693,0.0,0.0000,0.0,0.0,0.0,0.0084,0.0000,0.0000,0.0000,0.0000,0.0,0.0,0.0,142.8835,180.0,3.6333,3.6078,-2.5151,4,32.9,8,100.0000,93.9544,5497.5776,3,140.4,41N
1,16.914174,4.6396,11.865,95.4993,20.5092,0.0,191.3997,0.0,17.5335,0.0,0.0,0.0,0.1160,0.1328,0.1194,88.8889,0.0000,0.0,0.0,0.0,150.1400,180.0,11.2052,0.5329,-0.7941,1,38.6,10,100.0000,0.0000,889.5946,0,327.2,41N
2,16.914174,7.2531,10.640,92.9991,231.1212,0.0,428.0046,0.0,10.5472,0.0,0.0,0.0,4.4543,0.1544,0.4562,100.0000,0.0000,0.0,0.0,0.0,180.0000,180.0,5.1324,4.2151,-1.9970,1,58.3,12,92.9884,0.0000,1209.4880,0,410.9,41N
3,16.914174,9.4746,11.455,90.9543,195.4548,0.0,389.1873,0.0,9.2675,0.0,0.0,0.0,0.3935,0.1245,0.2198,89.2857,3.5714,0.0,0.0,0.0,180.0000,180.0,4.1777,2.7572,-4.8641,1,16.3,11,95.0483,0.0000,839.0543,0,329.2,41N
4,16.914174,10.8893,11.570,89.6720,115.9858,0.0,411.0793,0.0,10.6438,0.0,0.0,0.0,1.4789,0.1557,0.2076,100.0000,0.0000,0.0,0.0,0.0,173.5012,180.0,6.6170,4.0348,-3.8334,1,46.9,11,93.3853,0.0000,918.8754,0,337.2,41N


In [46]:
# Scamble and subset data frame into train + validation(80%) and test(10%)
df_feature_filtered = df_feature_filtered.sample(frac=1).reset_index(drop=True)
split_ratio = 0.8
df_train = df_feature_filtered[ : int(len(df_feature_filtered) * split_ratio)]
df_test = df_feature_filtered[int(len(df_feature_filtered) * split_ratio) : ]


In [20]:
from sklearn.model_selection import KFold
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report

In [21]:
kf = KFold(n_splits = 10)
solver = MLPClassifier(activation='relu', solver='adam', alpha=1e-5, hidden_layer_sizes=(100, 2), random_state=1, verbose=True)

# convert dataframe to ndarray, since kf.split returns nparray as index
feature_train = df_train.iloc[:, 0 : -1].values 
target_train = df_train.iloc[:, -1].values 
feature_test = df_test.iloc[:, 0 : -1].values 
target_test = df_test.iloc[:, -1].values 
#y_pred = cross_val_predict(solver, feature, target, cv = 10, verbose=True)

In [ ]:
for train_indices, test_indices in kf.split(feature_train, target_train):
    solver.fit(feature_train[train_indices], target_train[train_indices])
    print(solver.score(feature_train[test_indices], target_train[test_indices]))

Iteration 1, loss = 0.41448837
Iteration 2, loss = 0.16251090
Iteration 3, loss = 0.10014153
Iteration 4, loss = 0.07659786
Iteration 5, loss = 0.05231417
Iteration 6, loss = 0.04233815
Iteration 7, loss = 0.03087196
Iteration 8, loss = 0.02913821
Iteration 9, loss = 0.02690034
Iteration 10, loss = 0.02094786
Iteration 11, loss = 0.02224027
Iteration 12, loss = 0.01783935
Iteration 13, loss = 0.01856586
Iteration 14, loss = 0.01566739
Iteration 15, loss = 0.02295671
Iteration 16, loss = 0.01566431
Iteration 17, loss = 0.02143841
Training loss did not improve more than tol=0.000100 for two consecutive epochs. Stopping.
0.996046298419
Iteration 1, loss = 0.33854592
Iteration 2, loss = 0.16077212
Iteration 3, loss = 0.14849981
Iteration 4, loss = 0.14262902
Iteration 5, loss = 0.13810084
Iteration 6, loss = 0.13544299
Iteration 7, loss = 0.13391636
Iteration 8, loss = 0.13384566
Iteration 9, loss = 0.12968484


In [23]:
y_pred = solver.predict(feature_test)
accuracy_score(y_pred, target_test)

0.99665367866147148

In [24]:
#class_names = ['41N', '64E', '80', '81W', '92E']
print(classification_report(target_test, y_pred))

             precision    recall  f1-score   support

        41N       1.00      1.00      1.00       758
        64E       1.00      1.00      1.00     37409
         80       0.91      0.97      0.94      1147
        81W       0.99      0.96      0.98      2943
        92E       0.99      1.00      0.99      1373

avg / total       1.00      1.00      1.00     43630



In [25]:
unique, count = np.unique(target_test, return_counts=True)
print(np.asarray((unique, count)).T)


[['41N' 758]
 ['64E' 37409]
 ['80' 1147]
 ['81W' 2943]
 ['92E' 1373]]


In [26]:
unique, count = np.unique(y_pred, return_counts=True)
print(np.asarray((unique, count)).T)

[['41N' '758']
 ['64E' '37409']
 ['80' '1221']
 ['81W' '2855']
 ['92E' '1387']]


In [51]:
df_feature_filtered.head()

,2nd Color Variation (Dark Yellow +),Elongation,brix,Overall Roundness,Calyx size,seed_counts,Stem Size,Creases,Dark Orange,frost_damage,Lemon (Dark Yellow +),Light Green,Orange/Yellow,Red,Red/Orange,Rough skin,Ridges,Stem,Stem Area,Stem Rot,Stem angle,Trapeziod Angle (Deg),Flatness,Fruit Center X (mm),Fruit Center Y (mm),Grade,LR_Diff,SizerSize,Smoothness,Start of Batch,Texture,VisionGrade,Weight,block_Num
0,16.914174,4.2238,12.745,95.3603,27.6507,9.34,63.2979,0.0000,74.4551,27.955,0.0,0.0,0.0789,0.2342,0.1099,4.7619,0.0,0.0000,0.0,7.0290,166.2603,180.0,8.3052,3.6887,-3.1200,1,8.8,5,100.0000,0.0191,76.3707,0,126.9,64E
1,16.914174,10.1047,13.515,88.6944,0.0000,5.29,0.0000,4.7619,2.0651,37.320,0.0,0.0,6.5375,0.0000,0.0393,19.0476,0.0,0.0000,0.0,0.0000,180.0000,180.0,22.4737,3.6160,-2.7347,3,27.2,6,100.0000,0.0116,421.0338,2,127.5,64E
2,16.914174,5.1653,12.930,94.1569,0.0000,4.77,0.0000,5.8824,1.1603,29.220,0.0,0.0,10.3322,0.0000,0.0185,11.7647,0.0,0.0000,0.0,0.0000,180.0000,180.0,10.9088,2.8996,-3.1742,2,16.3,3,94.9113,0.2180,369.4083,1,65.0,64E
3,16.914174,4.7067,12.190,95.4038,2.3029,5.24,96.7572,10.5263,75.1671,31.300,0.0,0.0,0.4748,0.3406,0.0568,0.0000,0.0,104.7837,0.0,3.1680,169.8418,180.0,5.8496,4.3834,-2.0899,1,18.2,5,100.0000,0.0155,177.2315,0,118.4,64E
4,16.914174,5.6697,12.420,94.4865,1.9472,2.19,129.6294,0.0000,77.0052,26.910,0.0,0.0,0.1189,0.6603,0.0682,4.7619,0.0,119.8555,0.0,1.3931,174.8531,180.0,8.2207,4.9271,-2.8239,1,16.0,5,100.0000,0.0000,430.2316,0,139.3,64E


In [160]:
def train(df, split_ratio):
    # Scamble and subset data frame into train + validation(80%) and test(10%)
    df = df.sample(frac=1).reset_index(drop=True)
    split_ratio = 0.8
    df_train = df[ : int(len(df) * split_ratio)]
    df_test = df[int(len(df) * split_ratio) : ]
    
    kf = KFold(n_splits = 10)
    solver = MLPClassifier(activation='relu', solver='adam', alpha=1e-5, hidden_layer_sizes=(100, 5), random_state=1, verbose=True)

    # convert dataframe to ndarray, since kf.split returns nparray as index
    feature_train = df_train.iloc[:, 0 : -1].values 
    target_train = df_train.iloc[:, -1].values 
    feature_test = df_test.iloc[:, 0 : -1].values 
    target_test = df_test.iloc[:, -1].values 
    #y_pred = cross_val_predict(solver, feature, target, cv = 10, verbose=True)
    
    for train_indices, test_indices in kf.split(feature_train, target_train):
        solver.fit(feature_train[train_indices], target_train[train_indices])
        print(solver.score(feature_train[test_indices], target_train[test_indices]))
    
    y_pred = solver.predict(feature_test)
    accuracy_score(y_pred, target_test)
    
    print(classification_report(target_test, y_pred))
    
    unique, count = np.unique(target_test, return_counts=True)
    print(np.asarray((unique, count)).T)
    
    unique, count = np.unique(y_pred, return_counts=True)
    print(np.asarray((unique, count)).T)
    

In [161]:
from sklearn.utils import resample
# Down-sample class of "64E"
def downsample(df, majority_target):
    df_majority = df[df.block_Num== majority_target]
    df_minority = df[df.block_Num!= majority_target]

    # Downsample majority class to the max number of minority classes for fair play
    # MiniMax 
    samples = 0
    for x in df_minority.block_Num.value_counts():
        samples = max(samples, x)
        # print("x = {0}".format(x))
        # print("samples = {0}".format(samples))

    df_majority_downsampled = resample(df_majority, replace=False, n_samples=samples, random_state=123)
    df_downsampled = pd.concat([df_majority_downsampled, df_minority])
    print (df_downsampled.block_Num.value_counts())

    return df_downsampled

#### Once the model is trained by data from 2017, test on data from 2018

In [145]:
import pandas as pd

'''
This function truncates the data frame to remain meaningful features
-------------------
Parameters: data frame, year
Returns: truncated data frame
'''

def truncate(data_frame, year):
    if year == 2017:
        print("===== Start truncating features ======")
        data_frame = data_frame.rename(index=str, columns={"IQS 1 avg": "brix", "IQS 2 avg": "granulation",
                                        "IQS 3 avg": "seed_counts", "IQS 4 avg": "frost_damage"})

        df_feature = data_frame[['2nd Color Variation (Dark Yellow +)', 'Elongation', 'Fruit', 'brix',
                                'granulation', 'Orange', 'Calyx size', 'seed_counts', 'Overall Roundness',
                                'Stem Size',  # Fill with mean

                                'Creases', 'Dark Orange', 'frost_damage', 'Lemon (Dark Yellow +)',
                                'Light Green', 'Orange/Yellow', 'Red', 'Red/Orange', 'Rough skin',
                                'Ridges', 'Stem', 'Stem Area', 'Stem Rot',  # Fill with zero

                                'Stem angle', 'Trapeziod Angle (Deg)',  # Fill with 180

                                'DownWeight', 'Dark Yellow', 'Yellow','Light Yellow', 
                                'Green', 'Dark Green', 'Scar', 'Roundness', 'Vertical Diameter Stability', 
                                'Curvature', 'Cyclic Rod Number', 'Trip Rod', 'Flatness', 'Fruit Center X (mm)', 
                                'Fruit Center Y (mm)', 'Grade', 'LR_Diff', 'Major Diameter (mm)', 'SizerSize', 
                                'Smoothness', 'Start of Batch', 'Texture', 'UpWeight', 'VisionGrade', 'Volume', 
                                'Weight', 'block_Num'  # No Filling
                                ]]
        df_feature= df_feature.fillna(df_feature.mean()[:])
        # # Fill with mean
        # df_feature = df_feature.fillna(df_feature.mean()['2nd Color Variation (Dark Yellow +)':'Stem Size'])

        # # Fill with zero
        # df_feature.loc[:, 'Creases': 'Stem Rot'] = df_feature.loc[:, 'Creases': 'Stem Rot'].fillna(0)

        # # Fill with 180
        # df_feature.loc[:, 'Stem angle': 'Trapeziod Angle (Deg)'] = df_feature.loc[:, 'Stem angle': 'Trapeziod Angle (Deg)'].fillna(180)
    
    elif year == 2018:
        print("===== Start truncating features ======")
        df_feature = data_frame[['2nd Color Variation (Dark Yellow +)', # Fill with mean

                                'Lemon (Dark Yellow +)',
                                'Light Green', 'Stem Area',  # Fill with zero

                                'DownWeight', 'Dark Yellow', 'Yellow','Light Yellow', 
                                'Green', 'Dark Green', 'Scar', 'Roundness', 'Vertical Diameter Stability', 
                                'Curvature', 'Cyclic Rod Number', 'Trip Rod', 'Flatness', 'Fruit Center X (mm)', 
                                'Fruit Center Y (mm)', 'Grade', 'LR_Diff', 'Major Diameter (mm)', 'SizerSize', 
                                'Smoothness', 'Start of Batch', 'Texture', 'UpWeight', 'VisionGrade', 'Volume', 
                                'Weight', 'block_Num'  # No Filling
                                ]]
        df_feature= df_feature.fillna(df_feature.mean()[:])
        # # Fill with mean
        # df_feature = df_feature.fillna(df_feature.mean()['2nd Color Variation (Dark Yellow +)'])

        # # Fill with zero
        # df_feature.loc[:, 'Lemon (Dark Yellow +)': 'Stem Area'] = df_feature.loc[:, 'Lemon (Dark Yellow +)': 'Stem Area'].fillna(0)

    # Convert Grade str to integer : A -> 1, B -> 2
    df_feature.Grade = [ord(x) - 64 for x in df_feature.Grade]

    return df_feature

'''
This function reduce the positive and negative correlated features
---------
Parameter: data frame, year
Returns: Filtered data frame
'''

def reduce(df_feature, year):
    print("===== Start reducing features ======")
    # df_feature = truncate(data_frame, year)
    df_target = df_feature[['block_Num']]
    df_feature = df_feature.drop('block_Num', axis=1)
    # import pdb; pdb.set_trace();

    correlation_matrix = df_feature.astype('float64').corr()
    for col in correlation_matrix.columns:
        if correlation_matrix[col].between(0.8, 1.0, inclusive=False).any() or \
                correlation_matrix[col].between(-1.0, -0.8).any():
                
            # Drop row and column in the correlation matrix
            correlation_matrix = correlation_matrix.drop(col)
            correlation_matrix = correlation_matrix.drop(col, axis=1)


    df_feature_filtered = df_feature[correlation_matrix.columns]

    # Reuse the index of df_feature_filtered
    df_feature_filtered = pd.concat([df_feature_filtered, df_target], axis=1)

    print("===== Finish reducing features ======")

    return df_feature_filtered


In [153]:
df_2017 = pd.read_csv("./data_2017.csv")
df_2018 = pd.read_csv("./data_2018.csv")

In [154]:
df_2017 = truncate(df_2017, 2017)
# df_2017 = reduce(df_2017, 2017)
df_2018 = truncate(df_2018, 2018)

===== Start truncating features ======
===== Start truncating features ======


In [149]:
df_2017 = df_2017.filter(items = df_2018.columns)

In [155]:
df_2017.columns

Index(['2nd Color Variation (Dark Yellow +)', 'Elongation', 'Fruit', 'brix',
       'granulation', 'Orange', 'Calyx size', 'seed_counts',
       'Overall Roundness', 'Stem Size', 'Creases', 'Dark Orange',
       'frost_damage', 'Lemon (Dark Yellow +)', 'Light Green', 'Orange/Yellow',
       'Red', 'Red/Orange', 'Rough skin', 'Ridges', 'Stem', 'Stem Area',
       'Stem Rot', 'Stem angle', 'Trapeziod Angle (Deg)', 'DownWeight',
       'Dark Yellow', 'Yellow', 'Light Yellow', 'Green', 'Dark Green', 'Scar',
       'Roundness', 'Vertical Diameter Stability', 'Curvature',
       'Cyclic Rod Number', 'Trip Rod', 'Flatness', 'Fruit Center X (mm)',
       'Fruit Center Y (mm)', 'Grade', 'LR_Diff', 'Major Diameter (mm)',
       'SizerSize', 'Smoothness', 'Start of Batch', 'Texture', 'UpWeight',
       'VisionGrade', 'Volume', 'Weight', 'block_Num'],
      dtype='object')

In [156]:
df_2017 = downsample(df_2017, "64E")
df_2017.shape

64E    14213
81W    14213
92E     6785
80      5842
41N     3794
Name: block_Num, dtype: int64


(44847, 52)

In [162]:
train(df_2017, 0.8)

Iteration 1, loss = 4.19478302
Iteration 2, loss = 1.55654813
Iteration 3, loss = 1.53152891
Iteration 4, loss = 1.51446710
Iteration 5, loss = 1.50344703
Iteration 6, loss = 1.49660656
Iteration 7, loss = 1.49247007
Iteration 8, loss = 1.48999707
Iteration 9, loss = 1.48852077
Iteration 10, loss = 1.48762099
Iteration 11, loss = 1.48707376
Iteration 12, loss = 1.48673385
Iteration 13, loss = 1.48654719
Iteration 14, loss = 1.48641832
Iteration 15, loss = 1.48634828
Iteration 16, loss = 1.48630888
Iteration 17, loss = 1.48628405
Training loss did not improve more than tol=0.000100 for two consecutive epochs. Stopping.
0.317168338907
Iteration 1, loss = 4.18353191
Iteration 2, loss = 1.55659509
Iteration 3, loss = 1.53182359
Iteration 4, loss = 1.51497340
Iteration 5, loss = 1.50411300
Iteration 6, loss = 1.49738572
Iteration 7, loss = 1.49333395
Iteration 8, loss = 1.49093327
Iteration 9, loss = 1.48950476
Iteration 10, loss = 1.48864906
Iteration 11, loss = 1.48812778
Iteration 12, lo

/Users/michaelzhang/miniconda3/envs/py3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
